<a href="https://colab.research.google.com/github/dzakypr/MaBe-2025-Challenge/blob/main/MabeCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         236G   39G  197G  17% /
tmpfs            64M     0   64M   0% /dev
shm              84G  4.0K   84G   1% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
tmpfs            84G   13M   84G   1% /var/colab
/dev/sda1       242G   41G  201G  17% /kaggle/input
tmpfs            84G     0   84G   0% /proc/acpi
tmpfs            84G     0   84G   0% /proc/scsi
tmpfs            84G     0   84G   0% /sys/firmware
drive           236G   49G  188G  21% /content/drive


In [31]:
# %cd "/content/MaBe-2025-Challenge"
# !git add .
# !git commit -m "Adding stats.pt"
# !git push origin main

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
!git config --global user.email prastiadidzaky@gmail.com
!git config --global user.name dzakypr
!git clone https://{userdata.get('mabe-toke')}@github.com/dzakypr/MaBe-2025-Challenge.git

fatal: destination path 'MaBe-2025-Challenge' already exists and is not an empty directory.


In [34]:
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda12x

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [35]:
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import gc, re, math
import itertools
from tqdm.auto import tqdm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import json
from sklearn.model_selection import train_test_split
import pickle
import glob
import random

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.cuda.amp import autocast, GradScaler

sys.path.insert(1, "/content/MaBe-2025-Challenge/lib")
import data_handling
import feature_eng
import feature_eng_gpu

importlib.reload(data_handling)
importlib.reload(feature_eng)
importlib.reload(feature_eng_gpu)


<module 'feature_eng_gpu' from '/content/MaBe-2025-Challenge/lib/feature_eng_gpu.py'>

In [36]:
DIR = "/content/drive/MyDrive/data/MABe-mouse-behavior-detection/"
TRAIN_ANNOTATION = DIR + "train_annotation/"
TRAIN_TRACKING = DIR + "train_tracking/"

train = pd.read_csv(DIR + "train.csv")

# drop likely-sleeping MABe22 clips: condition == "lights on"
train = train.loc[~(train['lab_id'].astype(str).str.contains('MABe22', na=False) &
                    train['mouse1_condition'].astype(str).str.lower().eq('lights on'))].copy()

train['valid_ids'] = train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].apply(
    lambda x: [i + 1 for i, val in enumerate(x) if pd.notna(val)],
    axis=1
)

test = pd.read_csv(DIR + "test.csv")
test['sleeping'] = (
    test['lab_id'].astype(str).str.contains('MABe22', na=False) &
    test['mouse1_condition'].astype(str).str.lower().eq('lights on')
)
test['n_mice'] = 4 - test[['mouse1_strain','mouse2_strain','mouse3_strain','mouse4_strain']].isna().sum(axis=1)

body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

drop_body_parts = ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright',
                   'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright',
                   'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

body_parts = ['body_center',
 'ear_left',
 'ear_right',
 'headpiece_bottombackleft',
 'headpiece_bottombackright',
 'headpiece_bottomfrontleft',
 'headpiece_bottomfrontright',
 'headpiece_topbackleft',
 'headpiece_topbackright',
 'headpiece_topfrontleft',
 'headpiece_topfrontright',
 'lateral_left',
 'lateral_right',
 'neck',
 'nose',
 'tail_base',
 'tail_midpoint',
 'tail_tip',
 'hip_left',
 'hip_right',
 'head',
 'forepaw_left',
 'forepaw_right',
 'hindpaw_left',
 'hindpaw_right',
 'spine_1',
 'spine_2',
 'tail_middle_1',
 'tail_middle_2']

class_list = ['no_action',
 'allogroom',
 'approach',
 'attack',
 'attemptmount',
 'avoid',
 'biteobject',
 'chase',
 'chaseattack',
 'climb',
 'defend',
 'dig',
 'disengage',
 'dominance',
 'dominancegroom',
 'dominancemount',
 'ejaculate',
 'escape',
 'exploreobject',
 'flinch',
 'follow',
 'freeze',
 'genitalgroom',
 'huddle',
 'intromit',
 'mount',
 'rear',
 'reciprocalsniff',
 'rest',
 'run',
 'selfgroom',
 'shepherd',
 'sniff',
 'sniffbody',
 'sniffface',
 'sniffgenital',
 'submit',
 'tussle']

class_to_id = {label: idx for idx, label in enumerate(class_list)}

_sex_cols = [f'mouse{i}_sex' for i in range(1,5)]
_train_sex_lut = (train[['video_id'] + _sex_cols].drop_duplicates('video_id')
                  .set_index('video_id').to_dict('index'))
_test_sex_lut  = (test[['video_id']  + _sex_cols].drop_duplicates('video_id')
                  .set_index('video_id').to_dict('index'))
_FEATURE_TEMPLATES = {}

selected_body_parts = [item for item in body_parts if item not in drop_body_parts]

x_train, x_test = train_test_split(train, test_size=0.2, random_state=67)

print(f"Number of data: {len(train)}")
print(f"frequency of the number of mouse {train['valid_ids'].value_counts()}")
print(x_train['valid_ids'].value_counts(), x_test['valid_ids'].value_counts())
print(class_to_id)

Number of data: 8789
frequency of the number of mouse valid_ids
[1, 2, 3]       7933
[1, 2]           846
[1, 2, 3, 4]      10
Name: count, dtype: int64
valid_ids
[1, 2, 3]       6368
[1, 2]           655
[1, 2, 3, 4]       8
Name: count, dtype: int64 valid_ids
[1, 2, 3]       1565
[1, 2]           191
[1, 2, 3, 4]       2
Name: count, dtype: int64
{'no_action': 0, 'allogroom': 1, 'approach': 2, 'attack': 3, 'attemptmount': 4, 'avoid': 5, 'biteobject': 6, 'chase': 7, 'chaseattack': 8, 'climb': 9, 'defend': 10, 'dig': 11, 'disengage': 12, 'dominance': 13, 'dominancegroom': 14, 'dominancemount': 15, 'ejaculate': 16, 'escape': 17, 'exploreobject': 18, 'flinch': 19, 'follow': 20, 'freeze': 21, 'genitalgroom': 22, 'huddle': 23, 'intromit': 24, 'mount': 25, 'rear': 26, 'reciprocalsniff': 27, 'rest': 28, 'run': 29, 'selfgroom': 30, 'shepherd': 31, 'sniff': 32, 'sniffbody': 33, 'sniffface': 34, 'sniffgenital': 35, 'submit': 36, 'tussle': 37}


In [37]:
def load_process_output(data_path, lab_id, video_id, max_frames=None, action_map=None):
    tracking_path = os.path.join(data_path, 'train_annotation', lab_id, f'{video_id}.parquet')
    if not os.path.exists(tracking_path):
        return None
    df_events = pd.read_parquet(tracking_path)
    if df_events.empty:
        return pd.DataFrame(columns=['frame', 'agent_id', 'target_id', 'action'])
    if action_map:
        df_events = df_events.copy()
        df_events['action'] = df_events['action'].map(action_map)
        df_events = df_events.dropna(subset=['action'])
        df_events['action'] = df_events['action'].astype(int)
    starts = df_events['start_frame'].values
    stops = df_events['stop_frame'].values
    lengths = (stops - starts + 1).astype(int)
    all_frames = np.concatenate([
        np.arange(s, e + 1) for s, e in zip(starts, stops)
    ])
    agent_ids = np.repeat(df_events['agent_id'].values, lengths)
    target_ids = np.repeat(df_events['target_id'].values, lengths)
    actions = np.repeat(df_events['action'].values, lengths)
    df_dense = pd.DataFrame({
        'frame': all_frames,
        'agent_id': agent_ids,
        'target_id': target_ids,
        'action': actions
    })
    if max_frames:
        df_dense = df_dense[df_dense['frame'] < max_frames]

    return df_dense

In [39]:
common_universal_bodyparts_tracked = ['tail_base', 'ear_right', 'ear_left']
mov = ['body_center', 'nose', 'tail_base']
angle = [['nose', 'tail_base'], ['head', 'spine_1'],['forepaw_left', 'forepaw_right']]
dist = ['body_center', 'nose', 'tail_base']
dir = ['nose', 'tail_base']

In [41]:
def custom_loss(output, target, criterion):
    total_loss = 0
    for stage_output in output:
        total_loss += criterion(stage_output, target)
    return total_loss

In [42]:
datum = train.iloc[3]
print(datum['valid_ids'])
datum_labid = datum['lab_id']
datum_vidid = datum['video_id']
datum_fps = datum['frames_per_second']
datum_px_pr_cm = datum['pix_per_cm_approx']
datum_n_mice = datum['valid_ids']
x = data_handling.load_and_process_video(DIR, datum_labid, datum_vidid, datum_n_mice,datum_px_pr_cm, selected_body_parts, drop_body_parts)

[1, 2, 3]


In [43]:
def calculate_joint_stats(data, sample_ratio=0.1, save_path="stats.pt"):
    """
    Calculates Global Median and Joint IQR for coordinate data.

    Args:
        data_dir: Folder containing your .parquet tracking files.
        sample_ratio: Fraction of files to use for estimation (to save RAM).
        save_path: Where to save the stats dictionary.
    """

    # Randomly sample files if dataset is huge

    all_x = []
    all_y = []

    for i in tqdm(range(len(data))):
        try:
            datum = data.iloc[i]
            datum_labid = datum['lab_id']
            datum_vidid = datum['video_id']
            datum_fps = datum['frames_per_second']
            datum_px_pr_cm = datum['pix_per_cm_approx']
            datum_n_mice = datum['valid_ids']
            df = data_handling.load_and_process_video(DIR, datum_labid, datum_vidid, datum_n_mice,datum_px_pr_cm, selected_body_parts, drop_body_parts)

            if isinstance(df.columns, pd.MultiIndex):
                xs = df.xs('x', level='coords', axis=1).values.flatten()
                ys = df.xs('y', level='coords', axis=1).values.flatten()
            else:
                # Flat columns assumption: ends with '_x' or '_y'
                x_cols = [c for c in df.columns if c.endswith('_x') or '_x_' in c]
                y_cols = [c for c in df.columns if c.endswith('_y') or '_y_' in c]
                xs = df[x_cols].values.flatten()
                ys = df[y_cols].values.flatten()

            # Filter NaNs immediately
            all_x.append(xs[~np.isnan(xs)])
            all_y.append(ys[~np.isnan(ys)])

        except Exception as e:
            print(f"Skipping {f}: {e}")

    # Concatenate everything
    cat_x = np.concatenate(all_x)
    cat_y = np.concatenate(all_y)

    # 1. Calculate Medians (Centering)
    # Even if you centered per-video, this fixes any global bias
    median_x = np.median(cat_x)
    median_y = np.median(cat_y)

    # 2. Calculate Joint Scale (IQR)
    # We combine X and Y deviations to find a single "Pixel/CM Scale" factor
    dev_x = np.abs(cat_x - median_x)
    dev_y = np.abs(cat_y - median_y)
    combined_dev = np.concatenate([dev_x, dev_y])

    q25 = np.percentile(combined_dev, 25)
    q75 = np.percentile(combined_dev, 75)
    iqr = q75 - q25

    if iqr == 0: iqr = 1.0 # Safety

    stats = {
        'median_x': float(median_x),
        'median_y': float(median_y),
        'scale': float(iqr)
    }

    print("Stats Calculated:", stats)
    torch.save(stats, save_path)
    return stats

In [44]:
def build_agent_centric_training_sample(df_wide, df_dense_labels=None, max_neighbors=3, valid_ids=None):
    """
    Transforms coordinates into Agent-Centric inputs AND aligns labels for training.

    CRITICAL CHANGE: This version allows 'Self' to be a neighbor.
    This enables training on behaviors like 'Rear' or 'Groom' where target_id == agent_id.
    """
    # 1. Setup IDs
    present_ids = df_wide.columns.get_level_values('mouse_id').unique().tolist()
    if valid_ids:
        valid_ids = [int(v) for v in valid_ids]
        present_ids = [int(p) for p in present_ids]
        mice_ids = [m for m in present_ids if m in valid_ids]
    else:
        mice_ids = [int(p) for p in present_ids]
    mice_ids.sort()

    samples = []
    num_frames = len(df_wide)

    # Pre-process labels
    if df_dense_labels is not None and not df_dense_labels.empty:
        df_dense_labels['agent_id'] = df_dense_labels['agent_id'].astype(int)
        df_dense_labels['target_id'] = df_dense_labels['target_id'].astype(int)

    # 2. Pre-calculate Centroids
    mouse_centroids = {}
    for m in mice_ids:
        xs = df_wide[m].xs('x', level='coords', axis=1)
        ys = df_wide[m].xs('y', level='coords', axis=1)
        mouse_centroids[m] = (np.nanmedian(xs.values), np.nanmedian(ys.values))

    for ego_id in mice_ids:
        # --- A. Prepare Ego Features ---
        ego_df = df_wide[ego_id].sort_index(axis=1)
        ego_df = ego_df.interpolate(method='linear', limit=10, limit_direction='both')
        ego_data = ego_df.values

        # Mask Generation
        valid_counts = np.sum(~np.isnan(ego_data), axis=1)
        total_parts = ego_data.shape[1]
        valid_frames = valid_counts >= (total_parts * 0.5)
        nan_mask = torch.tensor(valid_frames, dtype=torch.float32).unsqueeze(0)

        # --- B. Sort Neighbors (INCLUDING SELF) ---
        distances = []
        for n_id in mice_ids:
            # REMOVED: if n_id == ego_id: continue  <-- This allows Self-Targeting

            e_c, n_c = mouse_centroids[ego_id], mouse_centroids[n_id]

            if np.isnan(e_c[0]) or np.isnan(n_c[0]):
                dist = 99999.0
            else:
                dist = np.sqrt((e_c[0]-n_c[0])**2 + (e_c[1]-n_c[1])**2)
            distances.append((n_id, dist))

        # Sort: Self (Dist 0) will always be first, followed by closest real neighbors
        distances.sort(key=lambda x: x[1])
        sorted_neighbors = [d[0] for d in distances]

        # --- C. Build Feature & Label Blocks ---
        neighbor_feature_blocks = []
        label_tensor = torch.zeros((max_neighbors, num_frames), dtype=torch.long)

        if df_dense_labels is not None and not df_dense_labels.empty:
            ego_acts = df_dense_labels[df_dense_labels['agent_id'] == ego_id]
        else:
            ego_acts = pd.DataFrame()

        for i in range(max_neighbors):
            if i < len(sorted_neighbors):
                n_id = sorted_neighbors[i]

                # 1. Neighbor Features
                n_df = df_wide[n_id].sort_index(axis=1)
                n_df = n_df.interpolate(method='linear', limit=10, limit_direction='both')
                neighbor_feature_blocks.append(n_df.values)

                # 2. Neighbor Labels
                if not ego_acts.empty:
                    target_acts = ego_acts[ego_acts['target_id'] == n_id]

                    if not target_acts.empty:
                        frames = target_acts['frame'].values
                        actions = target_acts['action'].values

                        valid_idx = frames < num_frames
                        frames = frames[valid_idx]
                        actions = actions[valid_idx]

                        if len(frames) > 0:
                            label_tensor[i, frames] = torch.tensor(actions, dtype=torch.long)

            else:
                # Padding
                neighbor_feature_blocks.append(np.zeros_like(ego_data))

        # --- D. Final Assembly ---
        all_data = np.concatenate([ego_data] + neighbor_feature_blocks, axis=1)
        all_data = np.nan_to_num(all_data, nan=0.0)

        feature_tensor = torch.tensor(all_data, dtype=torch.float32).T

        samples.append({
            'agent_id': ego_id,
            'features': feature_tensor,
            'labels': label_tensor,
            'mask': nan_mask,
            'neighbors': sorted_neighbors[:max_neighbors]
        })

    return samples

In [45]:
def prepare_and_save_dataset(
    metadata_df,
    DIR,
    output_dir="processed_dataset_chunks", # Changed: Saves to a directory now
    stats_path="stats.pt",
    max_neighbors=3,
    action_map=None
):
    """
    Full pipeline to load, transform, normalize, and save the dataset in CHUNKS.
    Saves individual .pkl files per video to prevent RAM crashes (OOM).

    Args:
        metadata_df: DataFrame containing video metadata (lab_id, video_id, valid_ids, etc.)
        data_dir: Root directory of the dataset.
        output_dir: Directory where processed chunks will be saved.
        stats_path: Path to normalization stats file.
        max_neighbors: Number of neighbor slots for the model.
        action_map: Dictionary mapping action strings to integers (1-based).
    """

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # 1. Load or Calculate Normalization Stats
    if os.path.exists(stats_path):
        print(f"Loading stats from {stats_path}...")
        stats = torch.load(stats_path)
    else:
        print("Calculating global stats...")
        # Note: You might want to adjust sample_ratio based on dataset size
        stats = calculate_joint_stats(metadata_df, data_dir, sample_ratio=0.1, save_path=stats_path)

    print(f"Normalization Stats: Median X={stats['median_x']:.2f}, Scale={stats['scale']:.2f}")

    # Removed: all_samples = [] (This caused the crash)

    print(f"Starting processing loop. Saving chunks to {output_dir}...")
    count = 0

    for i in tqdm(range(len(metadata_df))):
        try:
            datum = metadata_df.iloc[i]
            datum_labid = datum['lab_id']
            datum_vidid = datum['video_id']
            datum_fps = datum['frames_per_second']
            datum_px_pr_cm = datum['pix_per_cm_approx']
            datum_n_mice = datum['valid_ids']

            # Construct chunk filename
            chunk_path = os.path.join(output_dir, f"{datum_labid}_{datum_vidid}.pkl")

            # Optional: Skip if already exists (resume capability)
            # if os.path.exists(chunk_path): continue

            # --- A. Load Coordinates ---
            # We explicitly ask for 'nose' and 'tail_base' to ensure we have minimal features

            df_wide = data_handling.load_and_process_video(DIR,
                                                           datum_labid,
                                                           datum_vidid,
                                                           valid_ids=datum_n_mice,
                                                           add_bp=selected_body_parts,
                                                           drop_bp=drop_body_parts,
                                                           pixel_per_cm=datum_px_pr_cm,)

            if df_wide is None or df_wide.empty:
                continue

            # --- B. Load Annotations ---
            # Load sparse events and convert to dense frames
            df_sparse_annot = load_process_output(
                DIR, # This function likely constructs path internally
                datum_labid,
                datum_vidid,
                # No max_frames limit here, we handle it in building
                action_map=action_map
            )

            # Ensure dense labels (expand_events_to_frames logic needs to be here or inside load_process_output)
            # Assuming load_process_output ALREADY returns dense frames based on your previous snippet
            df_dense = df_sparse_annot

            # --- C. Apply Global Normalization (Manual Step) ---
            # We modify the dataframe in-place before transformation

            # Apply Scale (Divide)
            df_wide = df_wide / stats['scale']

            # Apply Centering (Subtract)
            df_wide.loc[:, (slice(None), slice(None), 'x')] -= stats['median_x']
            df_wide.loc[:, (slice(None), slice(None), 'y')] -= stats['median_y']

            # --- D. Build Agent-Centric Samples ---
            # This function handles Interpolation, Masking, Neighbor Sorting, and Label Alignment
            samples = build_agent_centric_training_sample(
                df_wide,
                df_dense_labels=df_dense,
                max_neighbors=max_neighbors,
                # Since we remapped IDs in load_video, valid IDs are now 1..N
                valid_ids=list(range(1, len(datum_n_mice) + 1))
            )

            # --- E. Save Chunk Immediately ---
            with open(chunk_path, 'wb') as f:
                pickle.dump(samples, f)

            count += len(samples)

            # --- F. Clear Memory ---
            del df_wide
            del samples
            del df_dense
            gc.collect()

        except Exception as e:
            print(f"Error processing {datum_vidid}: {e}")
            continue

    print(f"Processing complete. {count} total samples saved to {output_dir}")
    return output_dir

In [46]:
# x = prepare_and_save_dataset(x_test, DIR, output_dir='/content/drive/Shareddrives/Feature/test_data_chunks', stats_path='/content/MaBe-2025-Challenge/stats.pt',action_map=class_to_id)

In [47]:
class CoordinateDataset(Dataset):
    def __init__(self, chunk_dir=None, samples_list=None, drop_empty_ratio=0.0):
        """
        Args:
            chunk_dir: Path to folder containing .pkl chunks (Strategy 2)
            samples_list: Direct list of samples (if passed from memory)
        """
        self.samples = []

        if samples_list:
            self.samples = samples_list
        elif chunk_dir:
            print(f"Loading dataset chunks from {chunk_dir}...")
            files = sorted(glob.glob(os.path.join(chunk_dir, "*.pkl")))

            for f in tqdm(files):
                try:
                    with open(f, 'rb') as handle:
                        chunk_samples = pickle.load(handle)
                        self.samples.extend(chunk_samples)
                except Exception as e:
                    print(f"Error loading chunk {f}: {e}")

            if drop_empty_ratio > 0.0:
                print(f"Filtering dataset: Dropping {drop_empty_ratio*100:.1f}% of empty samples...")
            self.samples = self._filter_empty(self.samples, drop_empty_ratio)

        print(f"Final Dataset Size: {len(self.samples)} samples.")

    def _filter_empty(self, samples, ratio):
        kept_samples = []
        dropped_count = 0

        for s in samples:
            # Check if sample has ANY active label ( > 0 )
            # We check the 'labels' tensor.
            # Note: labels are (Neighbors, Time), usually integer types.
            if 'labels' in s:
                labels = s['labels']
                if isinstance(labels, torch.Tensor):
                    is_active = (labels > 0).any().item()
                else:
                    is_active = (np.array(labels) > 0).any()

                if is_active:
                    # Always keep interesting samples
                    kept_samples.append(s)
                else:
                    # It's an empty sample. Keep it with probability (1 - ratio)
                    if random.random() > ratio:
                        kept_samples.append(s)
                    else:
                        dropped_count += 1
            else:
                # No labels (inference mode), keep everything
                kept_samples.append(s)

        print(f"Dropped {dropped_count} empty samples.")
        return kept_samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # 1. Features
        features = sample['features']
        if not isinstance(features, torch.Tensor):
            features = torch.tensor(features, dtype=torch.float32)

        # 2. Labels
        if 'labels' in sample and sample['labels'] is not None:
            labels = sample['labels']
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.long)
        else:
            # Fallback for inference (no labels)
            # Assuming standard 3 neighbors
            num_neighbors = len(sample.get('neighbors', [0,0,0]))
            labels = torch.zeros((num_neighbors, features.shape[1]), dtype=torch.long)

        # 3. Mask
        if 'mask' in sample and sample['mask'] is not None:
            mask = sample['mask']
            if not isinstance(mask, torch.Tensor):
                mask = torch.tensor(mask, dtype=torch.float32)
        else:
            mask = torch.ones((1, features.shape[1]), dtype=torch.float32)

        return features, labels, mask

# ... (Keep the pad_collate and create_dataloader functions from previous response) ...
# They work exactly the same way.
def pad_collate(batch):
    # ... (Same as before) ...
    features, labels, masks = zip(*batch)
    lengths = [f.shape[1] for f in features]
    max_len = max(lengths)
    batch_size = len(features)
    n_feats = features[0].shape[0]
    n_neighbors = labels[0].shape[0]

    feat_pad = torch.zeros(batch_size, n_feats, max_len)
    label_pad = torch.zeros((batch_size, n_neighbors, max_len), dtype=torch.long)
    mask_pad = torch.zeros(batch_size, 1, max_len)

    for i in range(batch_size):
        l = lengths[i]
        feat_pad[i, :, :l] = features[i]
        label_pad[i, :, :l] = labels[i]
        mask_pad[i, :, :l] = masks[i]

    return feat_pad, label_pad, mask_pad

def create_dataloader(chunk_dir, batch_size=4, shuffle=True, num_workers=2, drop_empty_ratio=0.0):
    # Initialize using the directory
    ds = CoordinateDataset(chunk_dir=chunk_dir,drop_empty_ratio=drop_empty_ratio)
    loader = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=pad_collate
    )
    return loader

In [48]:
train_loader = create_dataloader(
    chunk_dir='/content/drive/Shareddrives/Feature/train_data_chunks', # The folder you saved to
    batch_size=4,
    drop_empty_ratio=0.85
)

Loading dataset chunks from /content/drive/Shareddrives/Feature/train_data_chunks...


  0%|          | 0/7031 [00:00<?, ?it/s]

Filtering dataset: Dropping 85.0% of empty samples...
Dropped 16753 empty samples.
Final Dataset Size: 3693 samples.


In [54]:
with open('/content/drive/Shareddrives/Feature/train.pkl', 'wb') as handle:
  pickle.dump(train_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [55]:
test_loader = create_dataloader(
    chunk_dir='/content/drive/Shareddrives/Feature/test_data_chunks', # The folder you saved to
    batch_size=4,
    drop_empty_ratio=0.85
)

Loading dataset chunks from /content/drive/Shareddrives/Feature/test_data_chunks...


  0%|          | 0/1758 [00:00<?, ?it/s]

Filtering dataset: Dropping 85.0% of empty samples...
Dropped 4105 empty samples.
Final Dataset Size: 980 samples.


In [56]:
with open('/content/drive/Shareddrives/Feature/test.pkl', 'wb') as handle:
  pickle.dump(test_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# for i in tqdm(range(len(train))):
#     datum = train.iloc[i]
#     datum_labid = datum['lab_id']
#     datum_vidid = datum['video_id']
#     datum_fps = datum['frames_per_second']
#     datum_px_pr_cm = datum['pix_per_cm_approx']
#     datum_n_mice = datum['valid_ids']
#     e = load_process_output(DIR, datum_labid, datum_vidid)
#     # print(e['action'].unique())
#     if e['action'].unique() == None:
#         print("yes")

In [57]:

train_loader.__len__()

924

In [58]:
def calculate_class_weights(dataloader, num_classes):
    """
    Calculates weights for CrossEntropyLoss based on frame counts per class.
    Run this ONCE before training using a DataLoader.

    Args:
        dataloader: A PyTorch DataLoader providing (features, labels, mask) batches.
        num_classes: Total number of classes (including No_Action).
    """
    counts = torch.zeros(num_classes)
    print("Scanning dataloader for class weights (Frame-based)...")

    # Iterate through batches
    for batch in tqdm(dataloader):
        _, labels, mask = batch
        # labels: (Batch, Neighbors, Time)
        # mask: (Batch, 1, Time)

        # Expand mask to match labels shape (Batch, Neighbors, Time)
        valid_mask = mask.expand_as(labels) > 0.5

        # Select only valid frames
        valid_labels = labels[valid_mask]

        # Accumulate counts
        unique, c = torch.unique(valid_labels, return_counts=True)
        for u, count in zip(unique, c):
            if u < num_classes:
                counts[u] += count

    print(f"Frame Counts: {counts.tolist()}")

    # Calculate Inverse Frequency Weights
    # weight = Total_Frames / (Num_Classes * Class_Count)
    # Add epsilon to counts to avoid div-by-zero if a class is missing
    total_frames = counts.sum()
    weights = total_frames / (num_classes * (counts + 1e-6))

    print(f"Calculated Weights: {weights.tolist()}")
    return weights

In [59]:
train_loader.__len__()

924

In [60]:
class_weights = calculate_class_weights(train_loader, len(class_to_id))

Scanning dataloader for class weights (Frame-based)...


  0%|          | 0/924 [00:00<?, ?it/s]

Frame Counts: [24466588.0, 0.0, 3778.0, 20675.0, 0.0, 8826.0, 0.0, 6192.0, 1180.0, 0.0, 0.0, 0.0, 0.0, 25348.0, 0.0, 0.0, 0.0, 1435.0, 0.0, 0.0, 29872.0, 0.0, 0.0, 22420.0, 0.0, 0.0, 46618.0, 12665.0, 0.0, 0.0, 0.0, 0.0, 21243.0, 0.0, 0.0, 33203.0, 3156.0, 0.0]
Calculated Weights: [0.026570281013846397, 650084155392.0, 172.07098388671875, 31.44300651550293, 650084155392.0, 73.65557861328125, 650084155392.0, 104.98775482177734, 550.9187622070312, 650084155392.0, 650084155392.0, 650084155392.0, 650084155392.0, 25.6463680267334, 650084155392.0, 650084155392.0, 650084155392.0, 453.02032470703125, 650084155392.0, 650084155392.0, 21.762325286865234, 650084155392.0, 650084155392.0, 28.995725631713867, 650084155392.0, 650084155392.0, 13.944917678833008, 51.329185485839844, 650084155392.0, 650084155392.0, 650084155392.0, 650084155392.0, 30.602275848388672, 650084155392.0, 650084155392.0, 19.579078674316406, 205.9835662841797, 650084155392.0]


# Model and its kwargs

In [64]:
model_args = {"mov1_channels": 128, "mov2_channels": 256, "mov1_dropout": 0.3, "mov2_dropout": 0.25,
              "mov1_layers": 0, "mov2_layers": 0,
              "pos1_channels": 32, "pos2_channels": 64, "pos1_dropout": 0.25, "pos2_dropout": 0.2,
              "pos1_layers": 0, "pos2_layers": 0,
              "tof1_channels": 256, "tof2_channels": 512, "tof1_dropout": 0.4, "tof2_dropout": 0.3,
              "tof1_layers": 0, "tof2_layers": 0,
              "lstm_hidden_size": 128, "gru_hidden_size": 128, "gaussian_noise_rate": 0.1, "dense_channels": 32,
              "cls_channels1": 256, "cls_dropout1": 0.2, "cls_channels2": 128, "cls_dropout2": 0.2,}

class SEBlock(nn.Module):
    def __init__(self, channels, reduction = 8):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, L)
        se = F.adaptive_avg_pool1d(x, 1).squeeze(-1)      # -> (B, C)
        se = F.relu(self.fc1(se), inplace=True)          # -> (B, C//r)
        se = self.sigmoid(self.fc2(se)).unsqueeze(-1)    # -> (B, C, 1)
        return x * se

class ResNetSEBlock(nn.Module):
    def __init__(self, in_channels, out_channels, wd = 1e-4):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels,
                               kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels,
                               kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        # SE
        self.se = SEBlock(out_channels)

        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          padding=0, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU(inplace=True)

class AttentionLayer(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.score_fn = nn.Linear(feature_dim, 1, bias=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # x: (B, L, F)
        score = torch.tanh(self.score_fn(x))     # (B, L, 1)
        weights = self.softmax(score.squeeze(-1))# (B, L)
        weights = weights.unsqueeze(-1)          # (B, L, 1)
        context = x * weights                    # (B, L, F)
        return context.sum(dim=1)                # (B, F)

class GaussianNoise(nn.Module):
    """Add Gaussian noise to input tensor"""
    def __init__(self, stddev):
        super().__init__()
        self.stddev = stddev

    def forward(self, x):
        if self.training:
            noise = torch.randn_like(x) * self.stddev
            return x + noise
        return x

class MaBeBackbone(nn.Module):
    def __init__(self, position_dim, movement_dim, distance_dim, angle_dim, **kwargs):
        super().__init__()
        self.movement = nn.Sequential(
            self.residual_feature_block(3, kwargs["mov1_channels"], kwargs["mov1_layers"], drop=kwargs["mov1_dropout"]),
            self.residual_feature_block(kwargs["mov1_channels"], kwargs["mov2_channels"], kwargs["mov2_layers"], drop=kwargs["mov2_dropout"])
        )
        self.distance = nn.Sequential(
            self.residual_feature_block(4, kwargs["mov1_channels"], kwargs["mov1_layers"], drop=kwargs["mov1_dropout"]),
            self.residual_feature_block(kwargs["mov1_channels"], kwargs["mov2_channels"], kwargs["mov2_layers"], drop=kwargs["mov2_dropout"])
        )
        self.angle = nn.Sequential(
            self.residual_feature_block(5, kwargs["mov1_channels"], kwargs["mov1_layers"], drop=kwargs["mov1_dropout"]),
            self.residual_feature_block(kwargs["mov1_channels"], kwargs["mov2_channels"], kwargs["mov2_layers"], drop=kwargs["mov2_dropout"])
        )

        self.pos_m_branch1 = self.init_pos_branch(position_dim//4, **kwargs)
        self.pos_m_branch2 = self.init_pos_branch(position_dim//4, **kwargs)
        self.pos_m_branch3 = self.init_pos_branch(position_dim//4, **kwargs)
        self.pos_m_branch4 = self.init_pos_branch(position_dim//4, **kwargs)

        self.some_proj = ResNetSEBlock(in_channels=3*kwargs["mov2_channels"], out_channels=kwargs["mov2_channels"])
        self.pos_proj = ResNetSEBlock(in_channels=5*kwargs["pos2_channels"], out_channels=kwargs["pos2_channels"])

        self.lstm = nn.LSTM(
            input_size=kwargs['mov2_channels']+kwargs['pos2_channels'],
            hidden_size=kwargs['lstm_hidden_size'],
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        self.gru = nn.GRU(
            input_size=kwargs['mov2_channels']+kwargs['pos2_channels'],
            hidden_size=kwargs['gru_hidden_size'],
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        self.noise = GaussianNoise(kwargs['gaussian_noise_rate'])
        self.dense = nn.Sequential(
            nn.Linear(kwargs['mov2_channels']+kwargs['pos2_channels'], kwargs['dense_channels']),
            nn.ELU()
        )

        self.attn = AttentionLayer(feature_dim=(kwargs['lstm_hidden_size']+kwargs['gru_hidden_size'])*2+kwargs['dense_channels'])  # lstm + gru + dense

    def feature_block(self, in_channels, out_channels, num_layers, pool_size=2, drop=0.3): # unchanged
        return nn.Sequential(
            *[ResNetSEBlock(in_channels=in_channels, out_channels=in_channels) for i in range(num_layers)],
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(pool_size, ceil_mode=True),
            nn.Dropout(drop)
        )

    def residual_feature_block(self, in_channels, out_channels, num_layers, pool_size=2, drop=0.3): # unchanged
        return nn.Sequential(
            *[ResNetSEBlock(in_channels=in_channels, out_channels=in_channels) for i in range(num_layers)],
            ResNetSEBlock(in_channels, out_channels, wd=1e-4),
            nn.MaxPool1d(pool_size, ceil_mode=True),
            nn.Dropout(drop)
        )

    def init_pos_branch(self, pos_dim , **kwargs):
        pos_branch = nn.Sequential(
            self.feature_block(pos_dim, kwargs["pos1_channels"], kwargs["pos1_layers"], drop=kwargs["pos1_dropout"]),
            self.feature_block(kwargs["pos1_channels"], kwargs["pos2_channels"], kwargs["pos2_layers"], drop=kwargs["pos2_dropout"]),
        )
        return pos_branch

    def forward(self, position, mov_angle_dist):
        mov, rot, dist = mov_angle_dist
        mov_feat = self.movement(mov.permute(0, 2, 1))
        rot_feat = self.dist(rot.permute(0, 2, 1))
        dist_feat = self.angle(dist.permute(0, 2, 1))
        main_feat = self.some_proj(torch.cat([mov_feat, rot_feat, dist_feat], dim=1))

        pos_m1, pos_m2, pos_m3, pos_m4 = position

        pos_m1_feat = self.pos_m_branch1(pos_m1.permute(0, 2, 1))
        pos_m2_feat = self.pos_m_branch2(pos_m2.permute(0, 2, 1))
        pos_m3_feat = self.pos_m_branch3(pos_m3.permute(0, 2, 1))
        pos_m4_feat = self.pos_m_branch4(pos_m4.permute(0, 2, 1))
        post_feat = self.pos_proj(torch.cat([pos_m1_feat, pos_m2_feat, pos_m3_feat, pos_m4_feat], dim=1))

        feat = torch.cat([main_feat, post_feat], dim=1).permute(0, 2, 1)
        lstm_out, _ = self.lstm(feat)
        gru_out, _ = self.gru(feat)
        dense_out = self.dense(self.noise(feat))

        return self.attn(torch.cat([lstm_out, gru_out, dense_out], dim=-1))

class MaBeModel(nn.Module):
    def __init__(self,position_dim, movement_dim, distance_dim, angle_dim, target_classes_num, **kwargs):
        super().__init__()
        self.backbone = MaBeBackbone(position_dim, movement_dim, distance_dim, angle_dim, **kwargs)
        self.target_classifier = nn.Sequential(
            nn.Linear((kwargs['lstm_hidden_size']+kwargs['gru_hidden_size'])*2+kwargs['dense_channels'], kwargs["cls_channels1"]),
            nn.BatchNorm1d(kwargs["cls_channels1"]),
            nn.ReLU(),
            nn.Dropout(kwargs["cls_dropout1"]),
            nn.Linear(kwargs["cls_channels1"], kwargs["cls_channels2"]),
            nn.BatchNorm1d(kwargs["cls_channels2"]),
            nn.ReLU(),
            nn.Dropout(kwargs["cls_dropout2"]),
            nn.Linear(kwargs["cls_channels2"], target_classes_num)
        )

    def forward(self, position, mov_angle_dist):
        feat = self.backbone(position, mov_angle_dist)
        targets_y = self.target_classifier(feat)
        return targets_y

In [65]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction = 8):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, L)
        se = F.adaptive_avg_pool1d(x, 1).squeeze(-1)      # -> (B, C)
        se = F.relu(self.fc1(se), inplace=True)          # -> (B, C//r)
        se = self.sigmoid(self.fc2(se)).unsqueeze(-1)    # -> (B, C, 1)
        return x * se

class ResNetSEBlock(nn.Module):
    def __init__(self, in_channels, out_channels, wd = 1e-4):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels,
                               kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels,
                               kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        # SE
        self.se = SEBlock(out_channels)

        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          padding=0, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU(inplace=True)

class AttentionLayer(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.score_fn = nn.Linear(feature_dim, 1, bias=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # x: (B, L, F)
        score = torch.tanh(self.score_fn(x))     # (B, L, 1)
        weights = self.softmax(score.squeeze(-1))# (B, L)
        weights = weights.unsqueeze(-1)          # (B, L, 1)
        context = x * weights                    # (B, L, F)
        return context.sum(dim=1)

class DilatedResidualLayer(nn.Module):
    def __init__(self, dilation, in_channel, out_channel) -> None:
        super(DilatedResidualLayer, self).__init__()
        self.conv_dilated = nn.Conv1d(in_channel, out_channel, 3, padding=dilation, dilation=dilation)
        self.conv1x1 = nn.Conv1d(in_channel, out_channel, 1)
        self.dropout = nn.Dropout()

    def forward(self, x):
        out = F.relu(self.conv_dilated(x))
        out = self.conv1x1(out)
        out = self.dropout(out)
        return x + out

class SingleStageModel(nn.Module):
    def __init__(self, num_layers, num_f_maps, dim, num_classes, num_neighbors=4):
        super(SingleStageModel, self).__init__()

        self.conv_1x1 = nn.Conv1d(dim, num_f_maps, 1)

        self.layers = nn.ModuleList([
            DilatedResidualLayer(2 ** i, num_f_maps, num_f_maps)
            for i in range(num_layers)
        ])
        self.conv_out = nn.Conv1d(num_f_maps, num_neighbors * num_classes, 1)

    def forward(self, x):
        out = self.conv_1x1(x)
        for layer in self.layers:
            out = layer(out)
        out = self.conv_out(out)
        return out


class MultiStageModel(nn.Module):
    def __init__(self, num_stages, num_layers, num_f_maps, dim, num_classes, num_neighbors):
        super(MultiStageModel, self).__init__()

        self.num_neighbors = num_neighbors
        self.num_classes = num_classes

        self.stage1 = SingleStageModel(num_layers, num_f_maps, dim, num_classes,num_neighbors)

        self.stages = nn.ModuleList([
            SingleStageModel(num_layers, num_f_maps, num_neighbors * num_classes, num_classes, num_neighbors)
            for _ in range(num_stages - 1)
        ])

    def forward(self, x, mask):
        out = self.stage1(x)
        outputs = [self._reshape_out(out)]
        for s in self.stages:
            B, _, T = out.shape
            temp = out.view(B, self.num_neighbors, self.num_classes, T)
            in_refinement = F.softmax(temp, dim=2) * mask.unsqueeze(1)
            out = s(in_refinement.view(B, -1,T))
            outputs.append(self._reshape_out(out))
        return outputs

    def _reshape_out(self, out):
        B,_,T = out.shape
        return out.view(B, self.num_neighbors, self.num_classes, T)

class MabeModel(nn.Module):
    def __init__(self, num_stages, num_layers, num_f_maps, dim, num_classes, num_neighbors):
        super(MabeModel, self).__init__()
        self.MultiStageModel = MultiStageModel(num_stages, num_layers, num_f_maps, dim, num_classes, num_neighbors)
        self.ResnetBlock = nn.Sequential()


class TMSELoss(nn.Module):
    def __init__(self, threshold=4.0):
        super(TMSELoss, self).__init__()
        self.threshold = threshold
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, predictions, batch_mask):
        loss = self.mse(torch.log_softmax(predictions[:, :, 1:], dim=1),
                        torch.log_softmax(predictions[:, :, :-1], dim=1))

        loss = torch.clamp(loss, min=0, max=self.threshold**2)

        mask = batch_mask[:, :, 1:]
        loss = torch.mean(loss * mask)

        return loss

# e

In [70]:
class TMSELoss(nn.Module):
    """
    Truncated Mean Squared Error Loss.
    Penalizes rapid flickering between classes (Smoothness constraint).
    """
    def __init__(self, threshold=4.0):
        super(TMSELoss, self).__init__()
        self.threshold = threshold
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, predictions, mask):
        # predictions shape: (Batch, Neighbors, Classes, Time)

        # We want to smooth along the Time dimension (last dim)
        # Compare frame T with frame T-1
        # Slice [..., 1:] is frames 1 to End
        # Slice [..., :-1] is frames 0 to End-1

        # Use Log Softmax for numerical stability
        log_probs = torch.log_softmax(predictions, dim=2)

        loss = self.mse(log_probs[:, :, :, 1:], log_probs[:, :, :, :-1])

        # Clamp the loss so we don't penalize HUGE changes (valid action transitions)
        # We only want to penalize "uncertain jitter"
        loss = torch.clamp(loss, min=0, max=self.threshold**2)

        # Apply mask to ignore padding (align mask to shape)
        # Mask is (Batch, 1, Time) -> Slice to (Batch, 1, Time-1)
        mask_sliced = mask[:, :, 1:]

        # Expand mask to match Neighbors/Classes dimensions
        # (Batch, 1, 1, Time-1) broadcasting
        mask_expanded = mask_sliced.unsqueeze(2)

        loss = loss * mask_expanded

        return loss.mean()

class ActionSegmentationLoss(nn.Module):
    """
    Combined Loss for MS-TCN.
    Calculates CrossEntropy + TMSE for EVERY stage output.
    """
    def __init__(self, class_weights=None, ignore_index=0, tmse_weight=0.15):
        """
        Args:
            class_weights: Tensor of weights for imbalanced classes.
            ignore_index: The label index to ignore (padding usually 0 or -100).
            tmse_weight: How much to prioritize smoothness (0.15 is standard).
        """
        super(ActionSegmentationLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights, reduction='none', ignore_index=ignore_index)
        self.tmse_loss = TMSELoss()
        self.tmse_weight = tmse_weight

    def forward(self, stage_outputs, targets, mask):
        """
        Args:
            stage_outputs: List of Tensors [Stage1_Out, Stage2_Out...]
                           Each shape: (Batch, Neighbors, Classes, Time)
            targets: Tensor (Batch, Neighbors, Time)
            mask: Tensor (Batch, 1, Time)
        """
        total_loss = 0

        # Flatten Batch and Neighbor dimensions for standard CrossEntropy
        # Target shape becomes (Batch * Neighbors, Time)
        B, N, T = targets.shape
        flat_targets = targets.view(-1, T)

        # Prepare Mask for CE (Batch * Neighbors, Time)
        flat_mask = mask.expand(B, N, T).reshape(-1, T)

        for out in stage_outputs:
            # 1. Classification Loss (Cross Entropy)
            # Input: (Batch, Neighbors, Classes, Time)
            # Reshape to (Batch * Neighbors, Classes, Time)
            C = out.shape[2]
            flat_out = out.view(B * N, C, T)

            # Calculate raw CE loss
            ce = self.ce_loss(flat_out, flat_targets)

            # Apply Time Mask manually (if ignore_index isn't enough)
            # ignore_index handles labels, but mask handles valid video duration
            masked_ce = (ce * flat_mask).sum() / (flat_mask.sum() + 1e-6)

            # 2. Smoothing Loss (TMSE)
            tmse = self.tmse_loss(out, mask)

            total_loss += masked_ce + (self.tmse_weight * tmse)

        return total_loss

In [71]:
import torch
import numpy as np
import datetime
import random
import copy
from torch.utils.tensorboard import SummaryWriter

class DeepLearningEssential(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)

        self.train_loader = None
        self.val_loader = None
        self.writer = None
        self.scheduler = None
        self.is_batch_lr_scheduler = False
        self.clipping = None

        self.losses = []
        self.val_losses = []
        self.learning_rates = []
        self.total_epochs = 0

        self.visualization = {}
        self.handles = {}

        self.train_step_fn = self._make_train_step_fn()
        self.val_step_fn = self._make_val_step_fn()

    def to(self, device):
        try:
            self.device = device
            self.model.to(self.device)
        except RuntimeError:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print(f"Couldn't send it to {device}, sending it to {self.device} instead.")
            self.model.to(self.device)

    def set_loaders(self, train_loader, val_loader=None):
        self.train_loader = train_loader
        self.val_loader = val_loader

    def set_tensorboard(self, name, folder="runs"):
        suffix = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer = SummaryWriter(f'{folder}/{name}_{suffix}')

    def _make_train_step_fn(self):
        def perform_train_step_fn(x, y, mask=None):
            self.model.train()

            # Handle mask for MS-TCN inputs
            if mask is not None:
                yhat = self.model(x, mask)
                loss = self.loss_fn(yhat, y, mask)
            else:
                yhat = self.model(x)
                loss = self.loss_fn(yhat, y)

            loss.backward()

            if self.clipping:
                self.clipping()

            self.optimizer.step()
            self.optimizer.zero_grad()

            return loss.item()
        return perform_train_step_fn

    def _make_val_step_fn(self):
        def perform_test_step_fn(x, y, mask=None):
            self.model.eval()

            if mask is not None:
                yhat = self.model(x, mask)
                loss = self.loss_fn(yhat, y, mask)
            else:
                yhat = self.model(x)
                loss = self.loss_fn(yhat, y)

            return loss.item()
        return perform_test_step_fn

    def _mini_batch_schedulers(self, frac_epoch):
        # Placeholder if not defined elsewhere
        if self.scheduler and self.is_batch_lr_scheduler:
            self.scheduler.step()

    def _epoch_schedulers(self, val_loss):
        # Placeholder if not defined elsewhere
        if self.scheduler and not self.is_batch_lr_scheduler:
            if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(val_loss)
            else:
                self.scheduler.step()

    def _mini_batch(self, validation=False):
        if validation:
            data_loader = self.val_loader
            step_fn = self.val_step_fn
        else:
            data_loader = self.train_loader
            step_fn = self.train_step_fn

        if data_loader is None:
            return None

        n_batches = len(data_loader)
        mini_batch_losses = []

        for i, batch in enumerate(data_loader):
            # Handle tuple unpacking for (features, labels, mask) or just (x, y)
            if len(batch) == 3:
                x_batch, y_batch, mask = batch
                x_batch = x_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                mask = mask.to(self.device)

                mini_batch_loss = step_fn(x_batch, y_batch, mask)
            else:
                x_batch, y_batch = batch
                x_batch = x_batch.to(self.device)
                y_batch = y_batch.to(self.device)

                mini_batch_loss = step_fn(x_batch, y_batch)

            mini_batch_losses.append(mini_batch_loss)

            if not validation:
                self._mini_batch_schedulers(i / n_batches)

        loss = np.mean(mini_batch_losses)
        return loss

    def set_seed(self, seed=42):
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        try:
            self.train_loader.sampler.generator.manual_seed(seed)
        except AttributeError:
            pass

    def train(self, n_epochs, seed=42, patience=None, min_delta=0.0):
        """
        Args:
            n_epochs: Total epochs to train.
            seed: Random seed.
            patience: Number of epochs to wait for improvement before stopping.
                      If None, early stopping is disabled.
            min_delta: Minimum change in validation loss to qualify as an improvement.
        """
        self.set_seed(seed)

        # Early Stopping State
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_state = None

        for epoch in range(n_epochs):
            self.total_epochs += 1

            loss = self._mini_batch(validation=False)
            self.losses.append(loss)

            with torch.no_grad():
                val_loss = self._mini_batch(validation=True)
                self.val_losses.append(val_loss)

            self._epoch_schedulers(val_loss)

            if self.writer:
                scalars = {'training': loss}
                if val_loss is not None:
                    scalars.update({'validation': val_loss})

                self.writer.add_scalars(main_tag='loss',
                                        tag_scalar_dict=scalars,
                                        global_step=epoch)

            # Print progress
            val_str = f" | Val Loss: {val_loss:.4f}" if val_loss is not None else ""
            print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {loss:.4f}{val_str}")

            # --- EARLY STOPPING LOGIC ---
            if patience is not None and val_loss is not None:
                # Check if validation loss improved by at least min_delta
                if val_loss < (best_val_loss - min_delta):
                    best_val_loss = val_loss
                    patience_counter = 0
                    # Save best state in memory
                    best_model_state = copy.deepcopy(self.model.state_dict())
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")
                        # Restore best model
                        if best_model_state:
                            self.model.load_state_dict(best_model_state)
                            print("Restored model weights from best epoch.")
                        break

        if self.writer:
            self.writer.close()

In [88]:
import torch.optim as optim

BATCH_SIZE = 4
TIME_STEPS = 1000 # 1000 frames (approx 30 seconds at 30fps)

# Using the numbers from your feature engineering
NUM_EGO_FEATS = len(selected_body_parts)*2 # for x and y
NUM_NEIGHBOR_FEATS = NUM_EGO_FEATS
MAX_NEIGHBORS = 3
NUM_CLASSES = len(class_list) # [No_Action, Attack, Sniff, Mount]

# Calculate total input channels (C)
# 26 + (24 * 3) = 98 channels
INPUT_CHANNELS = NUM_EGO_FEATS + (NUM_NEIGHBOR_FEATS * MAX_NEIGHBORS)

# --- 2. Instantiate the Model ---
modelTCN = MultiStageModel(
    num_stages=4,           # 4 stages (1 prediction + 3 refinement)
    num_layers=10,          # 10 layers per stage
    num_f_maps=64,          # Hidden dimension size
    dim=INPUT_CHANNELS,     # 98
    num_classes=NUM_CLASSES,# 4
    num_neighbors=MAX_NEIGHBORS # 3 output heads
)

loss_func = ActionSegmentationLoss(class_weights).to('cuda')
optimizer = optim.Adam(modelTCN.parameters(),lr=5e-4, weight_decay=1e-4)

# --- 3. Create Dummy Data (The Input) ---
# Input: (Batch, Channels, Time)
feat,label,mask = next(iter(train_loader))

print(f"Input Shape: {feat.shape}")
print(f"Mask Shape:  {mask.shape}")

# --- 4. Forward Pass ---
# The model returns a list of outputs (one for each stage)
# stage_outputs = model(feat, mask)

Input Shape: torch.Size([4, 128, 17119])
Mask Shape:  torch.Size([4, 1, 17119])


In [89]:
sigma = DeepLearningEssential(modelTCN, loss_func, optimizer=optimizer)

In [90]:
schedulere = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
)

sigma.set_loaders(train_loader,test_loader)
sigma.scheduler = schedulere

In [95]:
def clear_gpu_memory():
    """Safely clear PyTorch GPU memory."""
    try:
        gc.collect()  # Run Python garbage collector
        torch.cuda.empty_cache()  # Release unreferenced memory
        torch.cuda.ipc_collect()  # Collect inter-process memory
        print("✅ GPU memory cleared.")
    except Exception as e:
        print(f"⚠️ Error clearing GPU memory: {e}")

# Example usage after training
clear_gpu_memory()

✅ GPU memory cleared.


In [96]:
!nvidia-smi

Mon Dec 15 20:15:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             77W /  400W |    9031MiB /  81920MiB |    100%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [97]:
sigma.train(n_epochs=200, patience=7, min_delta=0.01)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.01 GiB. GPU 0 has a total capacity of 79.32 GiB of which 635.88 MiB is free. Process 18193 has 78.69 GiB memory in use. Of the allocated memory 76.65 GiB is allocated by PyTorch, and 1.55 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)